In [2]:
import numpy as np
import cv2
from keras.models import load_model

In [3]:
# Load the saved model
model = load_model('model_rafdb.h5')

In [4]:
# Emotion labels for FER 2013 dataset
emotion_labels = ['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral']

In [5]:
face_detection = cv2.CascadeClassifier('haar_cascade_face_detection.xml')

settings = {
	'scaleFactor': 1.3, 
	'minNeighbors': 5, 
	'minSize': (50, 50)
}

In [6]:
def preprocess_image(image_path, face_detection, settings):
    # Load image in color
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to load.")

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    detected_faces = face_detection.detectMultiScale(gray, **settings)

    if len(detected_faces) == 0:
        raise ValueError("No faces detected in the image.")

    # Process only the first detected face (optional: handle multiple faces)
    x, y, w, h = detected_faces[0]

    # Crop the detected face
    face = gray[y + 5:y + h - 5, x + 20:x + w - 20]
    # Resize face to 48x48
    face = cv2.resize(face, (48, 48))
    # Normalize pixel values
    face = face / 255.0
    # Expand dimensions to match model input shape (1, 48, 48, 1)
    face = np.expand_dims(face, axis=-1)  # Add channel dimension
    face = np.expand_dims(face, axis=0)  # Add batch dimension

    return face

In [7]:
def predict_emotion(image_path):
    try:
        # Preprocess the image
        processed_img = preprocess_image(image_path, face_detection, settings)

        # Validate processed image shape
        if processed_img is None or len(processed_img.shape) != 4:
            raise ValueError(f"Invalid processed image shape: {processed_img.shape if processed_img is not None else 'None'}")

        # Debugging shapes
        print(f"Processed image shape: {processed_img.shape}")
        print(f"Expected model input shape: {model.input_shape}")

        # Get prediction
        prediction = model.predict(processed_img)

        # Debugging prediction output
        print(f"Prediction output: {prediction}")

        # Get the index of the highest probability
        emotion_idx = np.argmax(prediction)
        # Get the corresponding label
        emotion = emotion_labels[emotion_idx]

        return emotion
    except Exception as e:
        print(f"Error during emotion prediction: {e}")
        return None

In [8]:
def predict_emotion_with_probabilities(image_path):
    try:
        # Preprocess the image
        processed_img = preprocess_image(image_path, face_detection, settings)
        
        if processed_img is None or len(processed_img.shape) != 4:
            raise ValueError(f"Invalid input shape: {processed_img.shape if processed_img is not None else 'None'}")

        # Print input shape for debugging
        print(f"Processed image shape: {processed_img.shape}")
        print(f"Model expected input shape: {model.input_shape}")

        # Get prediction probabilities for all classes
        predictions = model.predict(processed_img)[0]  # [0] to flatten the batch dimension
        
        # Get the index of the highest probability
        emotion_idx = np.argmax(predictions)
        # Get the corresponding label
        emotion = emotion_labels[emotion_idx]
        # Display probabilities for each class
        probabilities = {emotion_labels[i]: predictions[i] for i in range(len(emotion_labels))}
        return emotion, probabilities
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None

In [ ]:
image_path = 'test.jpg'
predicted_emotion, class_probabilities = predict_emotion_with_probabilities(image_path)

if predicted_emotion:
    print(f"The predicted emotion is: {predicted_emotion}")
    print("Class probabilities:")
    for emotion, prob in class_probabilities.items():
        print(f"{emotion}: {prob:.4f}")
else:
    print("Prediction failed.")

In [19]:
# Example usage
image_path = 'fearful (4).jfif'
predicted_emotion = predict_emotion(image_path)

if predicted_emotion:
    print(f"The predicted emotion is: {predicted_emotion}")
else:
    print("Failed to predict emotion.")

Error during emotion prediction: No faces detected in the image.
Failed to predict emotion.
